In [ ]:
import time
import sys
import os
import torch
import json
import pandas as pd
import numpy as np

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.torch.distributions import StudentTOutput
from gluonts.torch.distributions import NormalOutput
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

sys.path.append("../..")
from timefusion.utils import metrics

In [ ]:
# Import dataset
train_data = pd.read_csv("../../datasets/electricity/train.csv", index_col="date")
val_data = pd.read_csv("../../datasets/electricity/val.csv", index_col="date")
test_data = pd.read_csv("../../datasets/electricity/test.csv", index_col="date")

# Normalize the signal power of each column
stds = train_data.std()
train_data /= stds
val_data /= stds
test_data /= stds

In [ ]:
prediction_length = 24
freq = "H"

# Get training, validation and test dataset
train_flat = train_data.stack().reset_index()
train_flat.columns = ["date", "series", "value"]
train_dataset = PandasDataset.from_long_dataframe(train_flat, target="value",item_id="series",timestamp="date",freq=freq)

val_flat = val_data.stack().reset_index()
val_flat.columns = ["date", "series", "value"]
val_dataset = PandasDataset.from_long_dataframe(val_flat, target="value",item_id="series",timestamp="date",freq=freq)
val_dataset_14 = [PandasDataset.from_long_dataframe(val_flat.iloc[:-prediction_length*i*train_data.shape[1]] if i != 0 else val_flat, target="value",item_id="series",timestamp="date",freq=freq) for i in range(14)]

test_flat = test_data.stack().reset_index()
test_flat.columns = ["date", "series", "value"]
test_dataset_14 = [PandasDataset.from_long_dataframe(test_flat.iloc[:-prediction_length*i*train_data.shape[1]] if i != 0 else test_flat, target="value",item_id="series",timestamp="date",freq=freq) for i in range(14)]


In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_dataset_14[0],
    predictor=predictor,
    num_samples=128
)

realisations = list(ts_it)

In [ ]:
for i in test_data.iloc[-1]:
    print(i)

In [ ]:
len(realisations)

In [ ]:
# Train the model and make predictions
model = DeepAREstimator(
    prediction_length = prediction_length, 
    freq=freq,
    context_length = 1*prediction_length,
    num_layers = 4,
    hidden_size = 30,
    lr = 1e-4,
    dropout_rate = 0.01,
    distr_output = StudentTOutput(),
    trainer_kwargs={"max_epochs": 1}
)

In [ ]:
# Train model
predictor = model.train(training_data=train_dataset,validation_data=val_dataset,num_workers=3)

In [ ]:
# Validation test
samples = []
realisations = []

for dataset in test_dataset_14:
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=128
    )

    samples.append(list(forecast_it))
    realisations.append(list(ts_it))


In [ ]:
# Convert to tensors
sample_tensor = torch.tensor(np.array([[l.samples for l in s] for s in samples])).permute(0,2,1,4)
realisations_tensor = torch.tensor([np.array(r).squeeze()[:,-prediction_length:] for r in realisations])

In [ ]:
realisations_tensor.shape

In [ ]:
[np.array(r) for r in realisations]

In [ ]:
k = [[l.samples for l in s] for s in samples]

In [ ]:
np.array(k).shape

In [ ]:
samples[0]

In [ ]:
val_dataset_14[0]

In [ ]:
a = [1,2,3]
a[:-1]